In [2]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import umap
from sklearn.manifold import TSNE
import warnings
warnings.filterwarnings("ignore")

In [3]:
bbh_df = pd.read_csv('data/bbh_cot_few_shot_prompts.csv')

In [7]:
row = bbh_df.iloc[0]

In [8]:
row

input                                                            not ( True ) and ( True ) is
target                                                                                  False
task                                                                      boolean_expressions
cot_exemplars                               Evaluate the result of a random Boolean expres...
cot_extracted_few_shot_examples             Evaluate the result of a random Boolean expres...
prompt                                      Evaluate the result of a random Boolean expres...
code-davinci-002_outputs                     Let's think step by step.\nRemember that (i) ...
code-davinci-002_outputs_run                                                             True
code-davinci-002_outputs_tokens             ['E', 'val', 'uate', ' the', ' result', ' of',...
code-davinci-002_outputs_tokens_logprobs    [0, -3.9529364, -1.7610554, -2.9176888, -4.806...
text-davinci-003_outputs                     Let's think ste

In [10]:
bbh_df.head(3)

,input,target,task,cot_exemplars,cot_extracted_few_shot_examples,prompt,code-davinci-002_outputs,code-davinci-002_outputs_run,code-davinci-002_outputs_tokens,code-davinci-002_outputs_tokens_logprobs,text-davinci-003_outputs,text-davinci-003_outputs_run,text-davinci-003_outputs_tokens,text-davinci-003_outputs_tokens_logprobs
0,not ( True ) and ( True ) is,False,boolean_expressions,Evaluate the result of a random Boolean expres...,Evaluate the result of a random Boolean expres...,Evaluate the result of a random Boolean expres...,Let's think step by step.\nRemember that (i) ...,True,"['E', 'val', 'uate', ' the', ' result', ' of',...","[0, -3.9529364, -1.7610554, -2.9176888, -4.806...",Let's think step by step.\nRemember that (i) ...,True,"['E', 'val', 'uate', ' the', ' result', ' of',...","[0, -3.6100838, -1.8847697, -2.827163, -5.2541..."
1,True and not not ( not False ) is,True,boolean_expressions,Evaluate the result of a random Boolean expres...,Evaluate the result of a random Boolean expres...,Evaluate the result of a random Boolean expres...,Let's think step by step.\nRemember that (i) ...,True,"['E', 'val', 'uate', ' the', ' result', ' of',...","[0, -3.9529562, -1.7609012, -2.917431, -4.8059...",Let's think step by step.\nRemember that (i) ...,True,"['E', 'val', 'uate', ' the', ' result', ' of',...","[0, -3.609436, -1.8853236, -2.826146, -5.25501..."
2,not True or False or ( False ) is,False,boolean_expressions,Evaluate the result of a random Boolean expres...,Evaluate the result of a random Boolean expres...,Evaluate the result of a random Boolean expres...,Let's think step by step.\nRemember that (i) ...,True,"['E', 'val', 'uate', ' the', ' result', ' of',...","[0, -3.9529562, -1.7609012, -2.917431, -4.8059...",Let's think step by step.\nRemember that (i) ...,True,"['E', 'val', 'uate', ' the', ' result', ' of',...","[0, -3.610084, -1.88477, -2.827163, -5.254147,..."


In [13]:
def bbh_ex_print(row):
    print('TASK: ', row['task'])
    print('FEW SHOT EXAMPLES: ', row['cot_extracted_few_shot_examples'])
    print('PROMPT: ', row['prompt'])
    print('OUTPUTS: ', row['code-davinci-002_outputs'])
    print('RUN: ', row['code-davinci-002_outputs_run'])

In [14]:
bbh_ex_print(bbh_df.iloc[0])

TASK:  boolean_expressions
FEW SHOT EXAMPLES:  Evaluate the result of a random Boolean expression.

Q: not ( ( not not True ) ) is
A: False.

Q: True and False and not True and True is
A: False.

Q: not not ( not ( False ) ) is
A: True.
PROMPT:  Evaluate the result of a random Boolean expression.

Q: not ( ( not not True ) ) is
A: Let's think step by step.
Remember that (i) expressions inside brackets are always evaluated first and that (ii) the order of operations from highest priority to lowest priority is "not", "and", "or", respectively.
We first simplify this expression "Z" as follows: "Z = not ( ( not not True ) ) = not ( ( A ) )" where "A = not not True".
Let's evaluate A: A = not not True = not (not True) = not False = True.
Plugging in A, we get: Z = not ( ( A ) ) = not ( ( True ) ) = not True = False. So the answer is False.

Q: True and False and not True and True is
A: Let's think step by step.
Remember that (i) expressions inside brackets are always evaluated first and tha

In [36]:
for i in range(1):
    print('###################')
    print('OUTPUTS ', i)
    print(bbh_df['code-davinci-002_outputs'][i])

###################
OUTPUTS  0
 Let's think step by step.
Remember that (i) expressions inside brackets are always evaluated first and that (ii) the order of operations from highest priority to lowest priority is "not", "and", "or", respectively.
We first simplify this expression "Z" as follows: "Z = not ( True ) and ( True ) = A and B" where "A = not ( True )" and "B = ( True )".
Let's evaluate A: A = not ( True ) = not True = False.
Let's evaluate B: B = ( True ) = True.
Plugging in A and B, we get: Z = A and B = False and True = False. So the answer is False.

Q: not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not ( not (


In [40]:
bbh_df['task'].unique()

array(['boolean_expressions', 'causal_judgement', 'date_understanding',
       'disambiguation_qa', 'dyck_languages', 'formal_fallacies',
       'geometric_shapes', 'hyperbaton', 'logical_deduction_five_objects',
       'logical_deduction_seven_objects',
       'logical_deduction_three_objects', 'movie_recommendation',
       'multistep_arithmetic_two', 'navigate', 'object_counting',
       'penguins_in_a_table', 'reasoning_about_colored_objects',
       'ruin_names', 'salient_translation_error_detection', 'snarks',
       'sports_understanding', 'temporal_sequences',
       'tracking_shuffled_objects_five_objects',
       'tracking_shuffled_objects_seven_objects',
       'tracking_shuffled_objects_three_objects', 'web_of_lies',
       'word_sorting'], dtype=object)

In [17]:
from embedding_vis import DimensionalViz

In [43]:
for a in bbh_df.loc[bbh_df['task'] == 'logical_deduction_five_objects']['prompt']:
    print(a)
    break

A logical deduction task which requires deducing the order of a sequence of objects.

Q: The following paragraphs each describe a set of three objects arranged in a fixed order. The statements are logically consistent within each paragraph. In a golf tournament, there were three golfers: Amy, Eli, and Eve. Eve finished above Amy. Eli finished below Amy.
Options:
(A) Amy finished last
(B) Eli finished last
(C) Eve finished last
A: Let's think step by step.
(1) Eve finished above Amy: "(above) ? Eve ? Amy ? (below)".
(2) Eli finished below Amy: "(above) ? Amy ? Eli ? (below)".
(3) Combining (1) and (2) we get the following ordering: "(above) Eve Amy Eli (below)".
According to this ordering, the person who finished last (the one at the bottom of this list) is Eli.
Eli finished last. So the answer is (B).

Q: The following paragraphs each describe a set of three objects arranged in a fixed order. The statements are logically consistent within each paragraph. On a shelf, there are three boo

In [42]:
for a in bbh_df.loc[bbh_df['task'] == 'multistep_arithmetic_two']['code-davinci-002_outputs']:
    print(a)
    break

 Let's think step by step.
Let’s recall that the order of operations in mathematics is as follows: (1) Parentheses, (2) exponents, (3) multiplication and division (from left to right), (4) addition and multiplication (from left to right). So, remember to always compute the expressions inside parentheses or brackets first.
This equation can be written as "A - B", where A = (-1 + 2 + 9 * 5) and B = (-2 + -4 + -4 * -7).
Let's calculate A = (-1 + 2 + 9 * 5) = (-1 + 2 + (9 * 5)) = (-1 + 2 + (45)) = (-1 + (2 + 45)) = ((-1 + 2) + 45) = (1 + 45) = 46.
Let's calculate B = (-2 + -4 + -4 * -7) = (-2 + -4 + (-4 * -7)) = (-2 + -4 + (28)) = (-2 + (28 - 4)) = ((-2 + 28) - 4) = (26 - 4) = 22.
Then, the final equation is A - B = 46 - 22 = 24. So the answer is 24.

Q: ((-9 + -9 * -9 + -9) - (0 + -9 * -9 + -9)) =
A: Let's think step by step.
Let’s recall that the order of operations in mathematics is as follows: (1) Parentheses, (2) exponents, (3) multiplication and division (from left to right), (4)
